In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
import scipy.signal
import numpy as np
import pandas
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from matplotlib.pyplot import figure
import random
from heapq import nlargest
import math

In [ ]:
model = Sequential()
model.add(LSTM(20, input_shape = (200,10), return_sequences = True))
model.add(LSTM(20, return_sequences= True))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='RMSprop',metrics=['categorical_accuracy'])
model.summary()

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# data = pandas.read_csv('./data/Learning/MyData/Turning/version2/data6_7_8_9.csv')
data = pandas.read_csv('./data/Learning/MyData/Turning/rt_lt_classifier/data6_7.csv')


X = data[['magneticHeading','gyroX','gyroY','gyroZ','Yaw','Roll','Pitch','accx','accy','accz']]
target = data['TurnStatus']


##### 5-fold corss validation

##### fold 1 
# X_train = X[:120000]
# X_test = X[120000:135000]
# y_train = target[:120000]
# y_test = target[120000:135000]
######

##### fold 2
# X_train= X[:135000]
# X_test= X[135000:150000]
# y_train= target[:135000]
# y_test= target[135000:150000]
######

##### fold 3
# X_train= X[:150000]
# X_test= X[150000:165000]
# y_train= target[:150000]
# y_test= target[150000:165000]
######

##### fold 4
# X_train = X[:165000]
# X_test = X[165000:180000]
# y_train = target[:165000]
# y_test = target[165000:180000]
######

##### fold 5
X_train = X[:180000]
X_test = X[180000:]
y_train = target[:180000]
y_test = target[180000:]




# X_train = X[:40000]
# y_train = target[:40000]

# X_test = X[40000:]
# y_test = target[40000:]



In [ ]:
def SmoothData(data):
    size = data.shape[0] 
    dim = data.shape[1]
    
    smooth_data = np.zeros((size,dim))
    
    smooth_data[0] = data[0]
    smooth_data[1] = data[1]
    smooth_data[size-1] = data[size-1]
    smooth_data[size-2] = data[size-2]
    
    for i in range(2, size-2):
        smooth_data[i] = np.average([data[i], data[i-1], data[i+1]], axis=0)
    return smooth_data

In [ ]:
X_test = np.array(X_test)
X_train = np.array(X_train)
y_test = np.array(y_test)
y_train = np.array(y_train)

X_train = SmoothData(X_train)
X_test = SmoothData(X_test)

### one_hot encoding

In [ ]:
y_train_onehot  = []
y_test_onehot = []
for i in range(len(y_train)):
    if(y_train[i]==0):
        y_train_onehot.append(np.array([1,0,0]))
    elif (y_train[i]==1):
        y_train_onehot.append(np.array([0,1,0]))
    elif(y_train[i]==2):
        y_train_onehot.append(np.array([0,0,1]))
#     elif(y_train[i]==3):
#         y_train_onehot.append(np.array([0,0,0,1]))
        
for i in range(len(y_test)):
    if(y_test[i]==0):
        y_test_onehot.append(np.array([1,0,0]))
    elif (y_test[i]==1):
        y_test_onehot.append(np.array([0,1,0]))
    elif (y_test[i]==2):
        y_test_onehot.append(np.array([0,0,1]))
#     elif (y_test[i]==3):
#         y_test_onehot.append(np.array([0,0,0,1]))

In [ ]:
### shuffle the train data to decorelate the training window segment
X_train_shuffle = []
y_train_shuffle = []
shuffle_index = random.sample(range(0, 178800), 894)
for i in shuffle_index:
    X_train_shuffle.append (np.array([X_train[i:i+200]]).reshape(200,10))
    y_train_shuffle.append (np.array([y_train_onehot[i:i+200]]).reshape(200,3))

In [ ]:
history = model.fit(np.array(X_train_shuffle),
                    np.array(y_train_shuffle),
                    epochs=200,
                    validation_data = (np.array([X_train[i:i+200] for i in range(178800,179800,50)]),
                                      np.array([y_train_onehot[i:i+200] for i in range(178800,179800,50)])),
                    batch_size=64)


In [ ]:
results = model.predict(np.array(X_test[:16800]).reshape(84,200,10))
results = results.reshape(16800,3)

In [ ]:
len(X_test)

In [ ]:
result_class = []
for i in results:
    if(np.argmax(i)==0):
        result_class.append(0)
    elif(np.argmax(i)==1):
        result_class.append(1)
    elif(np.argmax(i)==2):
        result_class.append(2)

In [ ]:
"""
Utitlity Functions
"""
"""
Computing the energy (on y-axis)

First, scale the accelerometer signal to the 9.8 
Second, iterate list of events, calculate energy of the y-axis 
"""

def Energy(data):
    data = [abs(number) for number in data]
    return np.sum(np.power(data,2))

    
"""
Scaling turn signal to m/s2
"""
def Scale(data):
    scaled_data = data.copy()
    for i in range(len(scaled_data)):
        scaled_data[i] = scaled_data[i] * 9.8
    return scaled_data



In [ ]:
"""
Creating list of events and 
removing events with energy < 20 and duration lower than 30

"""
rt = 0
lt = 0
num_events = 0
events = []
freq = 100 
scaled_data = Scale([X_test[i][8] for i in range(len(X_test))]) # ay

for i in range(len(result_class)):
    if(result_class[i]==1):
        rt += 1
    elif(result_class[i]==2):
        lt += 1
    else:
        if(rt > 100 ):
            signal = [abs ((scaled_data)[i]) for i in range(i-rt, i-1)]
            energy = Energy(signal)
            if(energy > 10):
                events.append(("rt", i-rt, i-1, rt/freq))
                print(num_events, "- rt event," ,"duration:", rt/freq, " tsteps", "init:", i-rt ,"end:", i-1)
                num_events += 1
            rt = 0
        elif(lt > 100):
            signal = [abs ((scaled_data)[i]) for i in range(i-lt, i-1)]
            energy = Energy(signal)
            if(energy > 10):
                events.append(("lt", i-lt, i-1, lt/freq))
                print(num_events, "- lt event," ,"duration:", lt/freq, " tsteps", "init:", i-lt ,"end:", i-1)
                num_events += 1
            lt = 0

In [ ]:
def ComputeIntensity(signal):
    max_values = np.mean(nlargest(3, signal))
    if(max_values > 0.1 and max_values < 0.2):
        return 0
    elif(max_values > 0.2 and max_values< 0.4):
        return 1
    elif(max_values > 0.4):
        return 2  

In [ ]:
"""
Assign intensity to each event 
0, 1, 2 --> Low, Medium, High
thresholds:
    0.98 = 0.1G
    1.96 = 0.2G
    3.92 = 0.4G
"""    
i = 0
for e in events:
    event_type, init, end, duration = e
    signal = [abs(X_test)[i][8] for i in range(init, end)]
    events[i] += (ComputeIntensity(signal),)
    i +=1


In [ ]:
"""
merging neighbour events
"""
duration = 0
merge_index = 0
deletion_index = []
events.append(('',math.inf,math.inf,math.inf))

for i in range(0,len(events)-1):
    if(abs((events[i][2])-(events[i+1][1]))<11 and events[i][0]==events[i+1][0]):
        duration +=1
    else:
        init = events[i-duration][1]
        end = events[i][2]-1
        events.append((events[i-duration][0], init, end, end-init, events[i-duration][4]))
        deletion_index.append(np.arange(i-duration,i+1))
        duration = 0

In [ ]:
events.remove(('',math.inf,math.inf,math.inf))

In [ ]:
# deletion_index
del events[0:18]

In [ ]:
result_class = np.zeros(len(X_test))
for e in events:
    event_type, init, end, duration, intensity = e
    if(event_type!=''):
        if(event_type =="rt"):
            result_class[init:end] = 1
        elif(event_type=="lt"):
            result_class[init:end] = 2

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px


y_true = y_test.copy()
y_pred = result_class.copy()
classes = [0,1,2]
confusion_matrix = confusion_matrix(y_true, y_pred, labels=[0,1,2])
confusion_matrix
# fig = px.imshow(confusion_matrix)

# fig.show()

### Graph the result

In [ ]:

fig =tools.make_subplots(rows=1,cols=1)

result =  go.Scatter(
    x = list(range(0,16800)),
    y = result_class,
    mode = 'lines',
    name='model_result'
)

test =  go.Scatter(
    x = list(range(0,16800)),
    y = y_test,
    mode = 'lines',
    name='test'
)

fig.append_trace(result,1,1)
fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Turning Model Classification')


py.offline.plot(fig, filename='fold5_turning_model_rt_lt_v2')


### saving_model

In [ ]:
from keras.models import model_from_json
import os

# serialize model to JSON
model_json = model.to_json()
with open("./Learning_model/Turning/rt_lt_cv/v2/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./Learning_model/Turning/rt_lt_cv/v2/model.h5")
print("Saved model to disk")

### loading_model

In [ ]:
from keras.models import model_from_json
import os

# load json and create model
json_file = open('./Learning_model/Turning/rt_lt_cv/v2/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Learning_model/Turning/rt_lt_cv/v2/model.h5")
print("Loaded model from disk")
model = loaded_model

### Performance Calculator


In [ ]:
import numpy as np

"""
Precision = True predicted turnings / total turnings
Recall = True predicted turnings / total predicted turnings

F1 = ( 2 . precision . recall ) / precision + recall
performance = F1 * (IOU / #turning)
"""

fold1_precision = (7 / 5)
fold1_recall = (7 / 10)
fold1_f1 = (2 * fold1_precision * fold1_recall) / (fold1_precision + fold1_recall)
fold1_iou =  (85.0/644 + 383.0/533 + 380.0/504 + 254.0/505 + 187.0/505 + 313.0/502 + 448.0/546) / 5
fold1_perf = fold1_f1 * fold1_iou


########
fold2_precision = (5 / 5)
fold2_recall = (5 / 10 )
fold2_f1 = (2 * fold2_precision * fold2_recall) / (fold2_precision + fold2_recall)
fold2_iou =(451.0/706 + 371.0/572 + 420.0/641 + 352.0/699 + 310.0/435) / 5
fold2_perf = fold2_f1 * fold2_iou
########

########
fold3_precision = (9/ 9)
fold3_recall = (9 / 13 )
fold3_f1 = (2 * fold3_precision * fold3_recall) / (fold3_precision + fold3_recall)
fold3_iou = (304.0/569 + 621.0/730 + 416.0/472 + 116.0/379 + 
             124.0/379 +437.0 /650 + 286.0/380 + 395.0/521 + 251.0/465) / 9

fold3_perf = fold3_f1 * fold3_iou
########

########
fold4_precision = (4/ 3)
fold4_recall = (4 / 8 )
fold4_f1 = (2 * fold4_precision * fold4_recall) / (fold4_precision + fold4_recall)
fold4_iou = (305.0/419 + 437.0/599 + 288.0/499 + 119.0/499 ) /3

fold4_perf = fold4_f1 * fold4_iou
########



fold5_precision = (7 / 7)
fold5_recall = (7 / 11 )
fold5_f1 = (2 * fold5_precision * fold5_recall) / (fold5_precision + fold5_recall)
fold5_iou = (396.0/527 +  325.0/529 + 599.0/734 + 406.0/476 + 451.0/468 + 391.0/438+ 411.0/476) / 7
fold5_perf = fold5_f1 * fold5_iou
########





print("Performance:")
print(fold1_perf,fold2_perf,fold3_perf,fold4_perf,fold5_perf)
print("F1-score:")
print(fold1_f1,fold2_f1, fold3_f1,fold4_f1,fold5_f1)
print("IOU:")
print(fold1_iou,fold2_iou, fold3_iou,fold4_iou,fold5_iou)

print("performance mean:", np.mean([fold1_perf, fold2_perf, fold3_perf, fold4_perf, fold5_perf]))
print("f1-score mean:", np.mean([fold1_f1,fold2_f1,fold3_f1, fold4_f1,fold5_f1]))
print("iou mean:", np.mean([fold1_iou,fold2_iou, fold3_iou,fold4_iou,fold5_iou]))


### Graph the performance

In [ ]:
fig =tools.make_subplots(rows=1,cols=2)

f1 =  go.Bar(
    x = list(range(5)),
    y = [fold1_f1,fold2_f1,fold3_f1, fold4_f1,fold5_f1],
    width=[0.5,0.5,0.5,0.5,0.5],
    name='F1-score'
)

iou =  go.Bar(
    x = list(range(5)),
    y = [fold1_iou,fold2_iou, fold3_iou,fold4_iou,fold5_iou],
    width=[0.5,0.5,0.5,0.5,0.5],
    name='Intersection-Over-Union'
)

fig.append_trace(f1,1,1)
fig.append_trace(iou,1,2)
fig.update_traces(marker_line_width=1.5, opacity=0.75)

fig['layout'].update(height=500, title = 'Turning Model Classification Accuracy')


py.offline.plot(fig, filename='turning_model_rt_lt_v2_acc')
